# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [1]:
%pip install --upgrade --quiet langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.9 MB/s eta 0:00:00


In [2]:
!pip install --quiet langchain_core

In [3]:
!pip install --quiet ollama openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00


If you'd like to use LangSmith, uncomment the below:

In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](/docs/integrations/chat/), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](/docs/integrations/chat/ollama/).

In [5]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [7]:
!rm /content/myscript.sh
!echo '#!/bin/bash' > /content/myscript.sh
!echo 'ollama serve' >> /content/myscript.sh
# Make the script executable
!chmod +x /content/myscript.sh
!nohup /content/myscript.sh &

nohup: appending output to 'nohup.out'


In [20]:
# Download the model to Colab environment
!nohup ollama pull phi3:mini > ollama_emd_model.log 2>&1 &

# Start the server to run the model and interact with it
!nohup ollama run phi3:mini > ollama_model.log 2>&1 &


In [12]:
import ollama

ollama_response = ollama.chat(model='phi3:mini', messages=[
   {
     'role': 'system',
     'content': 'You are a helpful assistant with sound mathematical knowledge.',
   },
   {
     'role': 'user',
     'content': 'Explain to me the Pythagorean theorem.',
   },

])
# Printing out of the generated response
print(ollama_response['message']['content'])

 The Pythagorean Theorem is a fundamental principle in Euclidean geometry that states, when it comes to right-angled triangles (those having an angle of 90 degrees), there's a specific relationship between the lengths of their sides: specifically, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares on the other two sides.

Mathematically this can be represented as `a^2 + b^2 = c^2`, where 'c' represents the length of the hypotenuse, while 'a' and 'b' are the lengths of the triangle's remaining two sides. 

This theorem is named after the ancient Greek mathematician Pythagoras who first formulated it around the 5th century B.C., though evidence suggests that this relationship was known to cultures across various continents before him, such as the Babylonians and Indians among others. The beauty of this simple but powerful theorem lies not just in its wide-ranging applications (from physics to computer science), but also in ho

## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](/docs/how_to/custom_tools).

In [61]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y



@tool
def get_current_time() -> str:
    """Returns the current time."""
    from datetime import datetime
    return datetime.now().strftime("%H:%M:%S")

@tool
def calculate_area_of_rectangle(length: float, width: float) -> float:
    """Calculates the area of a rectangle."""
    return length * width

@tool
def get_word_count(text: str) -> int:
    """Counts the number of words in a string."""
    return len(text)

tools = [add, multiply, get_current_time, calculate_area_of_rectangle, get_word_count]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}
--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
get_current_time
Returns the current time.
{}
--
calculate_area_of_rectangle
Calculates the area of a rectangle.
{'length': {'title': 'Length', 'type': 'number'}, 'width': {'title': 'Width', 'type': 'number'}}
--
get_word_count
Counts the number of words in a string.
{'text': {'title': 'Text', 'type': 'string'}}


In [28]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [16]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [63]:
from langchain.schema.runnable import Runnable
import ollama

class OllamaLLM(Runnable):
    def __init__(self, model_name):
        self.model_name = model_name

    def invoke(self, input, config=None):
        # Handle ChatPromptValue object
        if hasattr(input, "to_string"):
            user_input = input.to_string()
        else:
            user_input = input.get("input", "")

        response = ollama.chat(model=self.model_name, messages=[
           {
                "role": "system",
                "content": """\
            You are an assistant that has access to the following set of tools.
            Here are the names, descriptions, and arguments for each tool:

            {rendered_tools}

            Given the user input, return the name and input of the tool to use. If there is no tool, use get_word_count as DEFAULT.
            Return your response as a JSON blob with 'name' and 'arguments' keys.

            The `arguments` should be a dictionary. The keys and types depend on the tool:
            - For tools like `add` and `multiply`, provide `x` and `y` as integers or floats.
            - For `get_current_time`, no arguments are needed.
            - For `calculate_area_of_rectangle`, provide `length` and `width` as floats.
            - For `get_word_count`, provide `text` as a string.

            Example:
            {
                "name": "multiply",
                "arguments": {"x": 3, "y": 5}
            }
            """
            },
            {
                'role': 'user',
                'content': user_input,
            },
        ])
        return response['message']['content']


In [64]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
User input: {input}

Example: "input": "what's 3 plus 5"

"""
)

# Instantiate the OllamaLLM with the desired model name
model = OllamaLLM(model_name='phi3:mini')

chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

 ```json
{
  "name": "add",
  "arguments": {"x": 3, "y": 1132}
}
```


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [50]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
result = chain.invoke({"input": "what's thirteen times 4"})
result

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}

In [51]:
chain = prompt | model | JsonOutputParser()
result = chain.invoke({"input": "what's the current time?"})
result

{'name': 'get_current_time', 'arguments': {}}

In [52]:
chain = prompt | model | JsonOutputParser()
result = chain.invoke({"input": "What's the are of a rectangle with sides of 6 and 4?"})
result

{'name': 'calculate_area_of_rectangle',
 'arguments': {'length': 6.0, 'width': 4.0}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [66]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [54]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

In [55]:
invoke_tool(result)

24.0

In [56]:
# Get Current Time
print("Current Time: ", invoke_tool({"name": "get_current_time", "arguments": {}}))  # Output: Current time (e.g., "15:30:22")

# Calculate Area of Rectangle
print("Area of Rectangle: ", invoke_tool({"name": "calculate_area_of_rectangle", "arguments": {"length": 4, "width": 6}}))  # Output: 24

# Get Word Count
print("Word Count: ", invoke_tool({"name": "get_word_count", "arguments": {"text": "This is a sentence."}}))  # Output: 5

Current Time:  16:06:59
Area of Rectangle:  24.0
Word Count:  4


## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [57]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})


53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [58]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)

chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

In [59]:
# Get Current Time
print("Current Time: ", chain.invoke({"input": "what's the current time?"}))

# Calculate Area of Rectangle
print("Area of Rectangle: ", chain.invoke({"input": "What's the are of a rectangle with sides of 6 and 4?"}))

# Get Word Count
print("Word Count: ", chain.invoke({"input": "Count the words in this sentence"}))

Current Time:  {'name': 'get_current_time', 'arguments': {}, 'output': '16:07:13'}
Area of Rectangle:  {'name': 'calculate_area_of_rectangle', 'arguments': {'length': 6, 'width': 4}, 'output': 24.0}
Word Count:  {'name': 'get_word_count', 'arguments': {'text': 'Count the words in this sentence.'}, 'output': 6}


In [65]:
chain_2 = prompt | model | JsonOutputParser() | invoke_tool

# Get Current Time
print("Current Time: ", chain_2.invoke({"input": "what's the current time?"}))

# Calculate Area of Rectangle
print("Area of Rectangle: ", chain_2.invoke({"input": "What's the are of a rectangle with sides of 6 and 4?"}))

# Get Word Count
print("Word Count: ", chain_2.invoke({"input": "Discuss the importance of tools in modern technology and innovation. How do tools empower individuals and industries alike? Explore the evolution of tools from simple implements to advanced technological aids. Consider examples from different fields such as construction, medicine, software development, and creative arts. Highlight the role of tools in improving efficiency, creativity, and problem-solving capabilities in today's world."}))

Current Time:  16:09:24
Area of Rectangle:  24.0
Word Count:  8


## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).